# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,972.729881,0.096063,1234,972.825943,972.633818,1945.651887,1945.267636
6,541.883865,-0.496314,1234,541.387551,542.380178,1082.775102,1084.760357
7,164.402801,-0.401677,1234,164.001124,164.804477,328.002249,329.608955
8,458.895014,-0.357374,1234,458.537640,459.252389,917.075280,918.504777
9,342.892982,0.493752,1234,343.386735,342.399230,686.773470,684.798460
10,579.342386,-0.768996,1234,578.573389,580.111382,1157.146779,1160.222765
11,268.222870,-0.738201,1234,267.484669,268.961071,534.969339,537.922143
12,574.368922,0.042074,1234,574.410995,574.326848,1148.821991,1148.653695
13,500.783882,-0.254356,1234,500.529526,501.038239,1001.059052,1002.076477
14,396.041559,0.685514,1234,396.727073,395.356045,793.454147,790.712091


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-972.825943,972.633818
6,-541.387551,542.380178
7,-164.001124,164.804477
8,-458.537640,459.252389
9,-343.386735,342.399230
10,-578.573389,580.111382
11,-267.484669,268.961071
12,-574.410995,574.326848
13,-500.529526,501.038239
14,-396.727073,395.356045


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-972.825943,972.633818
6,-541.387551,542.380178
7,-164.001124,164.804477
8,-458.537640,459.252389
9,-343.386735,342.399230
10,-578.573389,580.111382
11,-267.484669,268.961071
12,-574.410995,574.326848
13,-500.529526,501.038239
14,-396.727073,395.356045


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-972.82594334,  972.63381776],
       [-541.3875508 ,  542.3801783 ],
       [-164.00112445,  164.80447747],
       [-458.5376399 ,  459.25238856],
       [-343.38673489,  342.39922997],
       [-578.57338938,  580.11138235],
       [-267.48466928,  268.96107132],
       [-574.41099538,  574.32684764],
       [-500.52952608,  501.03823873],
       [-396.72707338,  395.35604527]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1943.651887,-1945.267636
6,1080.775102,-1084.760357
7,326.002249,-329.608955
8,915.075280,-918.504777
9,684.773470,-684.798460
10,1155.146779,-1160.222765
11,532.969339,-537.922143
12,1146.821991,-1148.653695
13,999.059052,-1002.076477
14,791.454147,-790.712091


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
